In [ ]:
## Custom all in one Data_generator

import cv2
import numpy as np
import pandas as pd

from torch.utils.data import Dataset

import torch
from torch import nn
from torchsummary import summary

import timm
from torch.optim import Adam, lr_scheduler


import SimpleITK as sitk

import os



class MyGen(Dataset):

    def __init__(self,
                 org_image_path = '/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset100_FracAtles/imagesTr',
                 pred_masks_path =  '/content/content/MSD/Dataset100_FracAtles/data_masks',
                 meta_data = final_csv):


        self.meta_data = meta_data


    def data_size(self):
        return self.meta_data.shape[0]

    def __len__(self):
        return self.meta_data.shape[0]


    def __getitem__(self, idx):

        # A path to a T1-weighted brain .nii image:
        org_img = org_image_path+ '/'+ self.meta_data['image_id'][idx]
        mask_img= pred_masks_path + '/' + self.meta_data['image_id'][idx][:-12] + '.nii.gz'
        # Read the .nii image containing the volume with SimpleITK:
        sitk_t1 = sitk.ReadImage(org_img)
        sitk_t2 = sitk.ReadImage(mask_img)
        # and access the numpy array:
        t1 = sitk.GetArrayFromImage(sitk_t1)
        t2 = sitk.GetArrayFromImage(sitk_t2)

        image = t1.transpose((2,1,0)).astype(np.float32)
        mask = t2.transpose((2,1,0)).astype(np.float32)
        seg_img = image + (mask * 40)

        seg_img = seg_img.transpose((2,0,1)).astype(np.float32)

        return seg_img

In [ ]:
import sys
import csv
import pickle
import numpy as np

import torch
from chexpert import CheXpert
from torchvision.models import densenet121
from torchvision.models import densenet161
import timm


gen_class =  MyGen(org_image_path = '/content/drive/MyDrive/Colab Notebooks/nnUNet_raw/Dataset100_FracAtles/imagesTr',  # Directory to the orginal images in nn-unet format ## add yours ##
              pred_masks_path =  '/content/content/MSD/Dataset100_FracAtles/data_masks', # Directory to the masks predicted nn-unet format ## add yours ##
              meta_data = final_csv,   #your dataframe variable should at least contian the 'image_id' column with image ids
                  ):
test_loader =  torch.utils.data.DataLoader(gen_class, batch_size=1, num_workers=1, shuffle=False)

# LOADING MODELS

model = timm.create_model('resnet18', num_classes=1, pretrained=False)
model = model.cuda()
model.load_state_dict('path_to_pth file')
model.eval()

with torch.no_grad():
    preds = []

    for test_1 in test_loader:
        test_1 = test_1.cuda()
        y_pred = model_CA(test_1)
        y_pred = torch.sigmoid(y_pred)
        y_pred = torch.round(y_pred)
        preds.append(y_pred.cpu().detach().numpy())
    preds = np.concatenate(preds)

final_csv['preds'] = preds

final_csv.to_csv('./final_results_csv')
